In [1]:
import sys
!{sys.executable} -m pip install python-dotenv

  Using cached python_dotenv-0.19.2-py2.py3-none-any.whl (17 kB)


In [2]:
from dotenv import dotenv_values

config = dotenv_values("../.env")

In [47]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import s3fs

import matplotlib.pyplot as plt
%matplotlib inline

import geoviews as gv
import hvplot.xarray
import hvplot.pandas
import hvplot.dask

import dask
from dask.distributed import Client
import dask_gateway

In [4]:
world_lowres = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
spain = world_lowres[world_lowres["name"] == "Spain"]
spain

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
132,48958159,Europe,Spain,ESP,1690000.0,"POLYGON ((-7.45373 37.09779, -7.53711 37.42890..."


In [5]:
gv.tile_sources.CartoLight() * gv.Polygons(spain.geometry)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [9]:
left, bottom, right, top = spain.geometry.total_bounds
left, bottom, right, top

(-9.392883673530648, 35.946850083961465, 3.0394840836805486, 43.74833771420099)

In [10]:
gateway = dask_gateway.Gateway()
cluster = gateway.new_cluster()

cluster.scale(6)
client = Client(cluster)
cluster

In [11]:
sfs = s3fs.S3FileSystem(
    anon=False,
    key="gregor",
    secret=config["fue_s3_key"],
    client_kwargs={
        "endpoint_url": "http://s3.fuecluster.hakom.at:9000",
    }
)
mapper = sfs.get_mapper("/energy-market-visual-exploration/era5_hourly_europe_2011_2021.zarr")

In [12]:
era5_europe = xr.open_zarr(mapper)
era5_europe

,Array,Chunk
Bytes,16.96 GiB,1.10 MiB
Shape,"(96432, 183, 258)","(720, 20, 20)"
Count,17421 Tasks,17420 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,16.96 GiB,1.10 MiB
Shape,"(96432, 183, 258)","(720, 20, 20)"
Count,17421 Tasks,17420 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [15]:
era5_europe.sel({"lon": slice(left, right), "lat": slice(top, bottom)})

<xarray.Dataset>
Dimensions:                      (valid_time_end_utc: 96432, lat: 31, lon: 50)
Coordinates:
  * lat                          (lat) float32 43.5 43.25 43.0 ... 36.25 36.0
  * lon                          (lon) float32 -9.25 -9.0 -8.75 ... 2.5 2.75 3.0
  * valid_time_end_utc           (valid_time_end_utc) datetime64[ns] 2011-01-...
Data variables:
    air_temperature_at_2_metres  (valid_time_end_utc, lat, lon) float32 dask.array<chunksize=(720, 12, 20), meta=np.ndarray>
    eastward_wind_at_10_metres   (valid_time_end_utc, lat, lon) float32 dask.array<chunksize=(720, 12, 20), meta=np.ndarray>
    northward_wind_at_10_metres  (valid_time_end_utc, lat, lon) float32 dask.array<chunksize=(720, 12, 20), meta=np.ndarray>
Attributes:
    institution:  ECMWF
    source:       Reanalysis
    tilte:        ERA5 forecasts

In [30]:
era5_aoi = era5_europe.sel({"lon": slice(left, right), "lat": slice(top, bottom), "valid_time_end_utc": slice("2019-01-01", "2021-12-31")})
windspeed = (era5_aoi["eastward_wind_at_10_metres"] ** 2 + era5_aoi["northward_wind_at_10_metres"] ** 2) ** 0.5
windspeed.attrs["units"] = era5_aoi["eastward_wind_at_10_metres"].attrs["units"]
era5_aoi["windspeed"] = windspeed
era5_aoi = era5_aoi.drop(["eastward_wind_at_10_metres", "northward_wind_at_10_metres"])
era5_aoi

<xarray.Dataset>
Dimensions:                      (valid_time_end_utc: 26304, lat: 31, lon: 50)
Coordinates:
  * lat                          (lat) float32 43.5 43.25 43.0 ... 36.25 36.0
  * lon                          (lon) float32 -9.25 -9.0 -8.75 ... 2.5 2.75 3.0
  * valid_time_end_utc           (valid_time_end_utc) datetime64[ns] 2019-01-...
Data variables:
    air_temperature_at_2_metres  (valid_time_end_utc, lat, lon) float32 dask.array<chunksize=(432, 12, 20), meta=np.ndarray>
    windspeed                    (valid_time_end_utc, lat, lon) float32 dask.array<chunksize=(432, 12, 20), meta=np.ndarray>
Attributes:
    institution:  ECMWF
    source:       Reanalysis
    tilte:        ERA5 forecasts

In [45]:
era5_aoi_local = era5_aoi.load()
era5_aoi_local

<xarray.Dataset>
Dimensions:                      (valid_time_end_utc: 26304, lat: 31, lon: 50)
Coordinates:
  * lat                          (lat) float32 43.5 43.25 43.0 ... 36.25 36.0
  * lon                          (lon) float32 -9.25 -9.0 -8.75 ... 2.5 2.75 3.0
  * valid_time_end_utc           (valid_time_end_utc) datetime64[ns] 2019-01-...
Data variables:
    air_temperature_at_2_metres  (valid_time_end_utc, lat, lon) float32 13.48...
    windspeed                    (valid_time_end_utc, lat, lon) float32 1.405...
Attributes:
    institution:  ECMWF
    source:       Reanalysis
    tilte:        ERA5 forecasts

In [46]:
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
nc_local_path = "../data/era_timeseries/spain_subset.nc"
era5_aoi_local.to_netcdf(nc_local_path, format="NETCDF4", engine="netcdf4")

In [55]:
client.close()
cluster.close()